In [137]:
import sys
sys.path.insert(0, "/Users/aimachine/PyImage/utils")
import numpy as np

import importlib
import matplotlib.pyplot as plt
import os
import glob
import skimage
from skimage import measure
from skimage import filters
from Normalize import normalizeFloat, normalizeMinMax, Path, save_tiff_imagej_compatible
from skimage.filters import threshold_otsu
from tifffile import imread
import scipy
from scipy import ndimage
from skimage.measure import compare_ssim as ssim
from bokeh.models import Label
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import export_png, output_notebook
from scipy import stats
from skimage.measure import compare_ssim as ssim

In [138]:
Lowbasedir = '/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/Low/'
GTbasedir = '/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/GT/'
Restoredbasedir = '/Users/aimachine/Downloads/GrantApplication/StructureSimilarity/Restored32Bit/'

In [139]:
LowPath = os.path.join(Lowbasedir, '*.tif')
GTPath = os.path.join(GTbasedir, '*.tif')
RestoredPath = os.path.join(Restoredbasedir, '*.tif')

filesLow = glob.glob(LowPath)
filesGT = glob.glob(GTPath)
filesRestored = glob.glob(RestoredPath)



In [140]:
print(len(filesLow))

76


In [141]:
def mse(x, y):
    return np.linalg.norm(x - y)

In [142]:

GTMSE = []
RestoredMSE = []

LowSNR = []
GTSNR = []
RestoredSNR = []
min = 0;
max = 1;



for fname in filesLow:
 Low = imread(fname) 
 Z = Low.shape[0]
 """Normalize"""
 Low = normalizeFloat(Low)
 for filenameGT in filesGT:
       
    if os.path.basename(filenameGT) ==  os.path.basename(fname): 
         GT = imread(filenameGT) 
         """Normalize"""
         GT = normalizeFloat(GT)
         filesGT.remove(filenameGT)   
            
 for filenameRestored in filesRestored:
  
        if os.path.basename(filenameRestored) == 'BorialisS1S2FlorisMidNoiseModel_' + os.path.basename(fname):         
          Restored = imread(filenameRestored) 
          """Normalize"""
          Restored = normalizeFloat(Restored)
          filesRestored.remove(filenameRestored) 
        


 for j in range(10, Z):
      smallLowimg = Low[j,:,:]
      smallGTimg = GT[j,:,:]
      smallRestoredimg = Restored[j,:,:]


     

    
      smallGTMSE = mse(smallGTimg, smallLowimg)
      smallLowssim = ssim(smallGTimg, smallLowimg, data_range=smallLowimg.max() - smallLowimg.min())
     
      
      GTMSE.append(smallGTMSE)
      LowSNR.append(smallLowssim)
     
        
      smallRestoredMSE = mse(smallGTimg, smallRestoredimg)
      smallRestoredssim = ssim(smallGTimg, smallRestoredimg, data_range=smallRestoredimg.max() - smallRestoredimg.min())
      RestoredMSE.append(smallRestoredMSE)
      RestoredSNR.append(smallRestoredssim)  
      

/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/site-packages/skimage/util/arraycrop.py:177: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


In [180]:
hist, edges = np.histogram(np.abs(GTMSE), density=False, bins='auto')
p = figure( y_axis_label='Counts',x_axis_label='MSE')
listedges = edges.tolist()
listhist = hist.tolist()

p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color="#043564", line_color="#643304")
sumFirst = 0
for  i in range(0,len(listhist)):
 if listhist[i] > 0:
  mytext = Label(x=listedges[i], y=listhist[i], text=str('%.2f'%listedges[i]))
  sumFirst = sumFirst + listhist[i]
  #p.add_layout(mytext)
output_notebook()    
show(p)

histSec, edgesSec = np.histogram(np.abs(RestoredMSE), density=False, bins='auto')
q = figure(y_axis_label='Counts',x_axis_label='MSE')
listedgesSec = edgesSec.tolist()
listhistSec = histSec.tolist()

q.quad(top=histSec, bottom=0, left=edgesSec[:-1], right=edgesSec[1:],
    fill_color="#036564", line_color="#033649")
sumSec = 0
for  i in range(0,len(listhistSec)):
 if listhistSec[i] > 0:
  mytext = Label(x=listedgesSec[i], y=listhistSec[i], text=str('%.2f'%listedgesSec[i]))
  sumSec = sumSec + listhistSec[i]
  #p.add_layout(mytext)
output_notebook()    
show(q)


q.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#043564", line_color="#643304")

show(q)
print(sumFirst, sumSec)


Loading BokehJS ...

Loading BokehJS ...

1216 1216


In [178]:
hist, edges = np.histogram(np.abs(LowSNR), density=False, bins='auto')
p = figure(y_axis_label='Counts',x_axis_label='SIM')
listedges = edges.tolist()
listhist = hist.tolist()

p.quad(top=histOne, bottom=0, left=edges[:-1], right=edges[1:],
     fill_color="#043564", line_color="#643304")
sumFirst = 0
for  i in range(0,len(listhist)):
 if listhist[i] > 0:
  mytext = Label(x=listedges[i], y=listhist[i], text=str('%.2f'%listedges[i]))
  sumFirst = sumFirst + listhist[i]
  #p.add_layout(mytext)

output_notebook()    
show(p)




histSec, edgesSec = np.histogram(np.abs(RestoredSNR), density=False, bins='auto')
q = figure(y_axis_label='Counts',x_axis_label='SIM')
listedgesSec = edgesSec.tolist()
listhistSec = histSec.tolist()

q.quad(top=histSec, bottom=0, left=edgesSec[:-1], right=edgesSec[1:],
    fill_color="#036564", line_color="#033649")
sumSec = 0
for  i in range(0,len(listhistSec)):
 if listhistSec[i] > 0:
  mytext = Label(x=listedgesSec[i], y=listhistSec[i], text=str('%.2f'%listedgesSec[i]))
  sumSec = sumSec + listhistSec[i]
  #p.add_layout(mytext)

output_notebook()    
show(q)

q.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
    fill_color="#043564", line_color="#643304")

show(q)

print(sumFirst, sumSec)


/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('right', 47), ('top', 15)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/site-packages/bokeh/models/sources.py:110: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('left', 47), ('right', 47), ('top', 15)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


Loading BokehJS ...

Loading BokehJS ...

1216 1216
